In [30]:
from sklearn.model_selection import cross_validate, LeaveOneGroupOut
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import networkx as nx


In [ ]:
# Populate the graph with edges and attributes
graphs = {}
for _, row in df.iterrows():
    # Check if graph_id exists, if not create a new DiGraph
    if row['graph_id'] not in graphs:
        graphs[row['graph_id']] = nx.DiGraph()
    G = graphs[row['graph_id']]
    
    # Add an edge if is_causal is True
    if row['is_causal']: 
        G.add_edge(row['edge_source'], row['edge_dest'])
    else:
        G.add_node(row['edge_source'])
        G.add_node(row['edge_dest'])


NameError: name 'df' is not defined

In [27]:
from graphviz import Digraph

for key, graph in graphs.items():
    G_dot = Digraph(engine="dot",format='png')

    for node in graph.nodes():
        G_dot.node(str(node))
    for edge in graph.edges():
        G_dot.edge(str(edge[0]), str(edge[1]))

    # Render the graph in a hierarchical layout
    G_dot.render(f"pics/graph_{key}.png", view=False)

In [63]:
#load dags.pkl
import pickle
with open('dags.pkl', 'rb') as f:
    dags = pickle.load(f)

    

In [29]:
from graphviz import Digraph

for key, graph in enumerate(dags):
    G_dot = Digraph(engine="dot",format='png')

    for node in graph.nodes():
        G_dot.node(str(node))
    for edge in graph.edges():
        G_dot.edge(str(edge[0]), str(edge[1]))

    # Render the graph in a hierarchical layout
    G_dot.render(f"pics/{key}_graph.png", view=False)

# Checking the Observations

# Reconstructing the formula

In [4]:
[(i, j) for i in range(3) for j in range(3)]

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [3]:
import numpy as np
np.round(np.random.uniform(low=-1, high=1),2)

0.35

In [31]:
import pickle

In [64]:
#load observations.pkl
with open('observations.pkl', 'rb') as f:
    observations = pickle.load(f)
    

In [33]:
import sys
sys.path.append("../d2c")
from utils import dag_to_formula

In [65]:
#load updated_dags.pkl
with open('updated_dags.pkl', 'rb') as f:
    updated_dags = pickle.load(f)

In [35]:
dag = updated_dags[0]


formula = dag_to_formula(dag)
print(formula)


1 = 0.09685*1_t-1 + -0.05417*1_t-2 + -0.40003*1_t-3 + 0
2 = -0.31657*1_t-1 + -0.31657*1_t-2 + -0.31657*1_t-3 + -0.04075*2_t-1 + -0.16629*2_t-2 + -0.35713*2_t-3 + 0
0 = 0.29654*1_t-1 + 0.29654*1_t-2 + 0.29654*1_t-3 + 0.27969*2_t-1 + 0.27969*2_t-2 + 0.27969*2_t-3 + 0.15089*0_t-1 + -0.44359*0_t-2 + 0.222*0_t-3 + 0



In [39]:
for value in observations[0].iloc[-1]:
    if abs(value) > 1000:
        

-0.0049
-0.0006200000000000003
-0.0014199999999999998


In [42]:
observations

[           0        1        2
 0    0.87711 -0.99844  0.98442
 1    0.23496  0.22331 -0.98587
 2   -0.95388  0.04955 -0.20028
 3   -0.32347  0.39022  0.05300
 4    0.30627 -0.05541  0.17074
 ..       ...      ...      ...
 147 -0.00117  0.00060  0.00347
 148 -0.00117  0.00148 -0.00129
 149  0.00503  0.00344  0.00258
 150  0.00617 -0.00037 -0.00551
 151 -0.00490 -0.00062 -0.00142
 
 [152 rows x 3 columns],
            0        1        2
 0   -0.85052 -0.90989  0.22426
 1    0.29725 -0.37010 -0.06969
 2   -0.10366 -0.08663  0.02100
 3    0.03506 -0.03533 -0.00471
 4   -0.01253 -0.00865  0.00015
 ..       ...      ...      ...
 145  0.00179  0.00042 -0.00068
 146 -0.00096 -0.00022  0.00154
 147  0.00037 -0.00245 -0.00035
 148 -0.00085 -0.00266  0.00112
 149  0.00117 -0.00133  0.00165
 
 [150 rows x 3 columns],
            0        1        2
 0   -0.48606  0.35985 -0.43488
 1    0.27771  0.04720  0.13777
 2   -0.12737  0.00526 -0.04318
 3    0.04766  0.00024  0.01287
 4   -0.01764 -0.0

In [47]:
exploding_series = []
for obs_index, observation in enumerate(observations):
    for value in observation.iloc[-1]:
        if abs(value) > 1000:
            exploding_series.append(obs_index)

        

In [48]:
exploding_series

[3, 29, 37, 38, 38, 38, 39, 39, 40, 40, 40]

In [66]:
# Using list comprehension to filter the elements
filtered_observations = [observations[i] for i in range(len(observations)) if i not in exploding_series]
filtered_dags = [dags[i] for i in range(len(dags)) if i not in exploding_series]
filtered_updated_dags = [updated_dags[i] for i in range(len(updated_dags)) if i not in exploding_series]


In [69]:
len(filtered_observations)

44

In [70]:
#pickle again 
with open('filtered_observations.pkl', 'wb') as f:
    pickle.dump(filtered_observations, f)

with open('filtered_dags.pkl', 'wb') as f:
    pickle.dump(filtered_dags, f)

with open('filtered_updated_dags.pkl', 'wb') as f:
    pickle.dump(filtered_updated_dags, f)



In [24]:
from statsmodels.tsa.api import VAR
import numpy as np
#np from txt
data = np.loadtxt('../causeme/experiments/linear-VAR_N-3_T-150_0198.txt')
# Fit the VAR model
model = VAR(data)
results = model.fit(maxlags=5, ic='aic')
print(results.params)


[[ 0.20921354  0.19466644  0.03685556]
 [ 0.80976437 -0.02165628  0.07613718]
 [ 0.1304932   0.95714291  0.04572878]
 [-0.06123438  0.02061111  0.39562886]
 [ 0.07480246  0.14377912  0.11043204]
 [-0.08990433 -0.13905322 -0.00875466]
 [ 0.19277196 -0.03335154 -0.07197485]]
